In [1]:
import pandas as pd

In [2]:
#Global paramenters
#API URLs from RxNorm
base_endpoint = "https://api.fda.gov/drug/ndc.json?search=active_ingredients.name:"
limit = "&limit=1000"
#dataframe
df = pd.DataFrame()

In [3]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    df = pd.read_json(base_endpoint+name_of_drug+limit, orient='values', typ='series', convert_dates=False)
    new_df = pd.DataFrame(df.results)
    return new_df

In [4]:
ingredients = ["alfentanil","benzhydrocodone","buprenorphine","butorphanol","codeine","dextromethorphan","difenoxin","dihydrocodeine","diphenoxylate","fentanyl","hydrocodone","hydromorphone","levorphanol","methadone","meperidine","morphine","nalbuphine","noscapine","oliceridine","opium","oxycodone","oxymorphone","pentazocine","remifentanil","sufentanil","tapentadol","tetrapeptide","tramadol"]

In [5]:
df_opioid = pd.DataFrame()
for drug in ingredients:
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, merg_df]).reset_index(drop=True)
df_opioid

,product_ndc,generic_name,labeler_name,dea_schedule,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,...,spl_id,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,brand_name_suffix,marketing_end_date
0,17478-067,Alfentanil Hydrochloride,Akorn,CII,Alfentanil Hydrochloride,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",True,"[{'package_ndc': '17478-067-02', 'description'...",20231231,"{'manufacturer_name': ['Akorn'], 'rxcui': ['17...",...,aea7d858-c41e-4726-8678-9c2bf08e5d56,HUMAN PRESCRIPTION DRUG,[INTRAVENOUS],20100201,17478-067_aea7d858-c41e-4726-8678-9c2bf08e5d56,NDA019353,Alfentanil Hydrochloride,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
1,49812-0033,Alfentanil HCl,"Veranova, L.P.",CII,NaN,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",False,"[{'package_ndc': '49812-0033-1', 'description'...",20231231,{},...,ef2a5a7a-737a-2610-e053-2995a90af9e7,BULK INGREDIENT,NaN,19970919,49812-0033_ef2a5a7a-737a-2610-e053-2995a90af9e7,NaN,None,NaN,NaN,NaN
2,67651-0337,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,NaN,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0337-0', 'description'...",20221231,{},...,405f5147-6d82-485f-8a67-906b294a282c,DRUG FOR FURTHER PROCESSING,NaN,20190628,67651-0337_405f5147-6d82-485f-8a67-906b294a282c,NaN,None,NaN,NaN,NaN
3,67651-0338,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,NaN,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0338-0', 'description'...",20221231,{},...,405f5147-6d82-485f-8a67-906b294a282c,DRUG FOR FURTHER PROCESSING,NaN,20190628,67651-0338_405f5147-6d82-485f-8a67-906b294a282c,NaN,None,NaN,NaN,NaN
4,67651-0339,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,CII,NaN,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",False,"[{'package_ndc': '67651-0339-0', 'description'...",20221231,{},...,405f5147-6d82-485f-8a67-906b294a282c,DRUG FOR FURTHER PROCESSING,NaN,20190628,67651-0339_405f5147-6d82-485f-8a67-906b294a282c,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,68025-072,tramadol hydrochloride,"Vertical Pharmaceuticals, LLC",CIV,ConZip,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '68025-072-30', 'description'...",20231231,{'manufacturer_name': ['Vertical Pharmaceutica...,...,bee9deef-8f3f-4f1c-b53b-b26e6842dc5e,HUMAN PRESCRIPTION DRUG,[ORAL],20110920,68025-072_bee9deef-8f3f-4f1c-b53b-b26e6842dc5e,NDA022370,ConZip,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3099,71335-0151,TRAMADOL HYDROCHLORIDE AND ACETAMINOPHEN,Bryant Ranch Prepack,CIV,TRAMADOL HYDROCHLORIDE AND ACETAMINOPHEN,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '71335-0151-1', 'description'...",20231231,{'manufacturer_name': ['Bryant Ranch Prepack']...,...,df3567f3-bfae-4c78-812f-35b234dfaa33,HUMAN PRESCRIPTION DRUG,[ORAL],20051216,71335-0151_df3567f3-bfae-4c78-812f-35b234dfaa33,ANDA077184,TRAMADOL HYDROCHLORIDE AND ACETAMINOPHEN,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3100,67296-1649,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",CIV,TRAMADOL HYDROCHLORIDE,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '67296-1649-1', 'description'...",20221231,"{'manufacturer_name': ['RedPharm Drug, Inc.'],...",...,b8df8cb5-10a1-58db-e053-2a95a90a8313,HUMAN PRESCRIPTION DRUG,[ORAL],20190101,67296-1649_b8df8cb5-10a1-58db-e053-2a95a90a8313,ANDA090404,TRAMADOL HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3101,68025-073,tramadol hydrochloride,"Vertical Pharmaceuticals, LLC",CIV,ConZip,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '68025-073-30', 'description'...",20231231,{'manufacturer_name': ['Vertical Pharmaceutica...,...,bee9deef-8f3f-

In [16]:
df_opioid.to_excel("Appendix16.xlsx", index=False)

In [7]:
#get within package
df_package = pd.DataFrame()
i = 0
while i < df_opioid.shape[0]:
    df_temp = pd.DataFrame(df_opioid.packaging[i])
    df_temp.insert(0,"generic_name", df_opioid.generic_name[i])
    df_temp.insert(1,"labeler_name", df_opioid.labeler_name[i])
    df_temp.insert(2,"brand_name", df_opioid.brand_name[i])
    df_package = pd.concat([df_package, df_temp]).drop_duplicates(subset=['package_ndc']).reset_index(drop=True)
    i+=1
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-02,10 AMPULE in 1 CARTON (17478-067-02) > 2 mL i...,20100201,False,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478-067-05,10 AMPULE in 1 CARTON (17478-067-05) > 5 mL i...,20100201,False,NaN
2,Alfentanil HCl,"Veranova, L.P.",NaN,49812-0033-1,1 CONTAINER in 1 CONTAINER (49812-0033-1) > 1...,19-SEP-97,NaN,NaN
3,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651-0337-0,18200 TABLET in 1 DRUM (67651-0337-0),28-JUN-19,NaN,NaN
4,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651-0338-0,18200 TABLET in 1 DRUM (67651-0338-0),28-JUN-19,NaN,NaN
...,...,...,...,...,...,...,...,...
6257,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296-1649-3,"30 TABLET, FILM COATED in 1 BOTTLE (67296-1649-3)",20190101,False,NaN
6258,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296-1649-6,"6 TABLET, FILM COATED in 1 BOTTLE (67296-1649-6)",20190101,False,NaN
6259,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296-1649-7,"15 TABLET, FILM COATED in 1 BOTTLE (67296-1649-7)",20190101,False,NaN
6260,tramadol hydrochloride,"Vertical Pharmaceuticals, LLC",ConZip,68025-073-30,"30 CAPSULE, EXTENDED RELEASE in 1 BOTTLE, PLAS...",20110920,False,NaN


In [8]:
#Change to NDC11
for ele in df_package.package_ndc:
    s = ele.split("-")
    S = []
    for i, L in enumerate([5,4,2]):
        if len(s[i]) < L:
            S.append('0'*(L-len(s[i])) + s[i])
        else:
            S.append(s[i])
    ndc11 = ''.join(S)
    df_package.replace(ele,ndc11, inplace=True)
df_package

,generic_name,labeler_name,brand_name,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006702,10 AMPULE in 1 CARTON (17478-067-02) > 2 mL i...,20100201,False,NaN
1,Alfentanil Hydrochloride,Akorn,Alfentanil Hydrochloride,17478006705,10 AMPULE in 1 CARTON (17478-067-05) > 5 mL i...,20100201,False,NaN
2,Alfentanil HCl,"Veranova, L.P.",NaN,49812003301,1 CONTAINER in 1 CONTAINER (49812-0033-1) > 1...,19-SEP-97,NaN,NaN
3,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651033700,18200 TABLET in 1 DRUM (67651-0337-0),28-JUN-19,NaN,NaN
4,BENZHYDROCODONE HYDROCHLORIDE and ACETAMINOPHEN,Patheon Pharmaceuticals Inc.,NaN,67651033800,18200 TABLET in 1 DRUM (67651-0338-0),28-JUN-19,NaN,NaN
...,...,...,...,...,...,...,...,...
6257,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296164903,"30 TABLET, FILM COATED in 1 BOTTLE (67296-1649-3)",20190101,False,NaN
6258,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296164906,"6 TABLET, FILM COATED in 1 BOTTLE (67296-1649-6)",20190101,False,NaN
6259,TRAMADOL HYDROCHLORIDE,"RedPharm Drug, Inc.",TRAMADOL HYDROCHLORIDE,67296164907,"15 TABLET, FILM COATED in 1 BOTTLE (67296-1649-7)",20190101,False,NaN
6260,tramadol hydrochloride,"Vertical Pharmaceuticals, LLC",ConZip,68025007330,"30 CAPSULE, EXTENDED RELEASE in 1 BOTTLE, PLAS...",20110920,False,NaN


In [9]:
df_package.to_excel("ndc_opioids.xlsx", index=False)

In [10]:
df_cdc = pd.read_excel("CDC_opioid NDC_oral MME conversion_update_2020.xlsx", sheet_name="Opioids", dtype='object')
df_cdc

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
15294,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
15295,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
15296,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
15297,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [11]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_package['package_ndc']:
    if df_cdc.isin([str(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1867
New drug count:  4395


In [12]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_cdc['NDC']:
    if df_package.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_cdc.loc[df_cdc['NDC']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1867


,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13427,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13428,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13429,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13430,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [13]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)
absent_drugs.to_excel("Absent_Opioid.xlsx", index=False)

In [14]:
absent_drugs

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13427,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13428,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13429,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13430,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
